# Imports

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import Normalizer
from copy import deepcopy
import operator as op
import warnings

import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import DataLoader, TensorDataset
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}.')

print ('Setup completed.')

Device: cuda.
Setup completed.


# Plan

## План-капкан состоит в следующем:
1. Загружаем оба train (train.csv) и test (test.csv) и дальше обрабатываем train и test одинаково и одновременно, чтобы потом не мучиться;
2. Загружаем metro (passenger_traffic_at_moscow_metro_stations.csv) и en_ru (en_ru.csv) и круто join-им их с train и test, в процессе убрав неинтересующие нас признаки;
3. Для тех станций метро, которые не нашлись в metro (и, соответственно, en_ru), заполняем все пустоты медианными значениями в соответствующих столбцах;
4. Делаем **умопомрачительный one-hot-encoding** всех категориальных признаков;
5. Удаляем выбросы, причём только из train;
6. Нормализуем (только истинно числовые признаки);
7. Копипастим нейронку и всё к ней прилагающееся (в т.ч. цикл обучения) с моего ноутбука по MNIST;
8. Делаем предсказания для test-а, создаём submission-ы и submit-им их.

# Work

## 1

In [2]:
train = pd.read_csv('/kaggle/input/flat-rent-prediction/train.csv', index_col='Unnamed: 0')
train.head()

,metro,price,minutes,way,provider,fee_percent,views,storey,storeys,rooms,total_area,living_area,kitchen_area
3257,Bulvar Rokossovskogo,35000,4,transport,agency,50,9,6,9,2,40,30,7
3822,Kuntcevskaia,59000,19,walk,agency,0,26,17,23,1,64,64,64
1263,Butyrskaia,47000,13,walk,agency,0,26,7,12,1,36,20,10
3575,Studencheskaia,110000,6,walk,agency,0,4,5,6,3,91,60,15
4074,No data,650000,0,walk,agency,0,12,65,74,3,64,64,64


In [3]:
test_X = pd.read_csv('/kaggle/input/flat-rent-prediction/test.csv', index_col='Unnamed: 0')
test_X.head()

,metro,minutes,way,provider,fee_percent,views,storey,storeys,rooms,total_area,living_area,kitchen_area
1350,Kommunarka,11,walk,agency,60,7,7,15,2,58,35,11
1434,No data,0,walk,agency,20,32,10,14,1,64,64,64
2500,Salarevo,16,transport,agency,100,35,11,16,1,64,64,64
1128,Prospekt Vernadskogo,16,walk,developer,50,19,1,7,2,58,36,9
4145,No data,0,walk,agency,0,16,2,12,7,64,64,64


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3392 entries, 3257 to 860
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   metro         3392 non-null   object
 1   price         3392 non-null   int64 
 2   minutes       3392 non-null   int64 
 3   way           3392 non-null   object
 4   provider      3392 non-null   object
 5   fee_percent   3392 non-null   int64 
 6   views         3392 non-null   int64 
 7   storey        3392 non-null   int64 
 8   storeys       3392 non-null   int64 
 9   rooms         3392 non-null   object
 10  total_area    3392 non-null   int64 
 11  living_area   3392 non-null   int64 
 12  kitchen_area  3392 non-null   int64 
dtypes: int64(9), object(4)
memory usage: 371.0+ KB


In [5]:
test_X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 849 entries, 1350 to 1427
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   metro         849 non-null    object
 1   minutes       849 non-null    int64 
 2   way           849 non-null    object
 3   provider      849 non-null    object
 4   fee_percent   849 non-null    int64 
 5   views         849 non-null    int64 
 6   storey        849 non-null    int64 
 7   storeys       849 non-null    int64 
 8   rooms         849 non-null    object
 9   total_area    849 non-null    int64 
 10  living_area   849 non-null    int64 
 11  kitchen_area  849 non-null    int64 
dtypes: int64(8), object(4)
memory usage: 86.2+ KB


In [6]:
train['metro'] = train['metro'].map(op.methodcaller('strip'))
test_X['metro'] = test_X['metro'].map(op.methodcaller('strip'))
print('done.')

done.


In [7]:
train_no_data = sum(train['metro'] == 'No data')
test_X_no_data = sum(test_X['metro'] == 'No data')
print(f'\'No data\' in \'metro\' in train: {train_no_data} = {train_no_data / len(train) * 100:.2f}% of the whole train.')
print(f'\'No data\' in \'metro\' in test_X: {test_X_no_data} = {test_X_no_data / len(test_X) * 100:.2f}% of the whole test_X.')

'No data' in 'metro' in train: 690 = 20.34% of the whole train.
'No data' in 'metro' in test_X: 159 = 18.73% of the whole test_X.


### Как неожиданно и приятно...

In [8]:
storey_to_floor = {'storey': 'floor', 'storeys': 'total_floors'}
train = train.rename(columns=storey_to_floor)
test_X = test_X.rename(columns=storey_to_floor)
print('done.')

done.


In [9]:
train['rooms'] = train['rooms'].replace('+', '2').astype(int)
test_X['rooms'] = test_X['rooms'].replace('+', '2').astype(int)
print('done.')

done.


## 2

In [10]:
metro = pd.read_csv('/kaggle/input/passenger-traffic-at-moscow-metro-stations/passenger_traffic_at_moscow_metro_stations.csv', sep=';')
metro.head()

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,NameOfStation,Line,Year,Quarter,IncomingPassengers,OutgoingPassengers,global_id,Unnamed: 7
0,Станция метрополитена,Линия,Год,Квартал,Входы пассажиров,Выходы пассажиров,global_id,NaN
1,Митино,Арбатско-Покровская линия,2021,I квартал,1913498,1829031,1138975996,NaN
2,Волоколамская,Арбатско-Покровская линия,2021,I квартал,1236714,1222309,1138975997,NaN
3,Строгино,Арбатско-Покровская линия,2021,I квартал,1938816,1903731,1138975999,NaN
4,Крылатское,Арбатско-Покровская линия,2021,I квартал,1849616,1818208,1138976000,NaN


In [11]:
metro = metro.drop(columns=['Line', 'global_id', 'Unnamed: 7']).iloc[1:]
metro.head()

,NameOfStation,Year,Quarter,IncomingPassengers,OutgoingPassengers
1,Митино,2021,I квартал,1913498,1829031
2,Волоколамская,2021,I квартал,1236714,1222309
3,Строгино,2021,I квартал,1938816,1903731
4,Крылатское,2021,I квартал,1849616,1818208
5,Площадь Революции,2021,I квартал,2324687,2319343


In [12]:
metro['Year'].unique()

array(['2021', '2022', '2023', '2024', '2025'], dtype=object)

In [13]:
metro['Quarter'].unique()

array(['I квартал', 'II квартал', 'III квартал', 'IV квартал'],
      dtype=object)

In [14]:
quarter_to_int = dict(zip(metro['Quarter'].unique(), range(4)))
period = metro['Year'].astype(int) * 4 + metro['Quarter'].map(quarter_to_int)
period -= period.min()

metro['period'] = period
metro = metro.drop(columns=['Year', 'Quarter'])
metro

,NameOfStation,IncomingPassengers,OutgoingPassengers,period
1,Митино,1913498,1829031,0
2,Волоколамская,1236714,1222309,0
3,Строгино,1938816,1903731,0
4,Крылатское,1849616,1818208,0
5,Площадь Революции,2324687,2319343,0
...,...,...,...,...
5529,Новаторская,1167230,1181704,18
5530,Новомосковская,171144,172445,18
5531,Академическая,74501,75067,18
5532,ЗИЛ,50797,51183,18


In [15]:
metro.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5533 entries, 1 to 5533
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   NameOfStation       5533 non-null   object
 1   IncomingPassengers  5533 non-null   object
 2   OutgoingPassengers  5533 non-null   object
 3   period              5533 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 173.0+ KB


In [16]:
metro['p_income'] = metro['IncomingPassengers'].astype(int)
metro['p_outcome'] = metro['OutgoingPassengers'].astype(int)
metro = metro.drop(columns=['IncomingPassengers', 'OutgoingPassengers'])
print('done.')

done.


In [17]:
en_ru = pd.read_csv('/kaggle/input/passenger-traffic-at-moscow-metro-stations/en_ru.csv')
en_ru.head()

,en,ru
0,Aeroport,Аэропорт
1,Akademicheskaia,Академическая
2,Alekseevskaia,Алексеевская
3,Alma-Atinskaia,Алма-Атинская
4,Altufevo,Алтуфьево


In [18]:
en_ru = en_ru.rename(columns={'en': 'metro', 'ru': 'NameOfStation'})
en_ru.head()

,metro,NameOfStation
0,Aeroport,Аэропорт
1,Akademicheskaia,Академическая
2,Alekseevskaia,Алексеевская
3,Alma-Atinskaia,Алма-Атинская
4,Altufevo,Алтуфьево


In [19]:
# здесь заджойним по максимуму, чтобы было больше данных для обучения
train = train.merge(en_ru, how='left').merge(metro, how='left').drop(columns='NameOfStation')
train.head()

,metro,price,minutes,way,provider,fee_percent,views,floor,total_floors,rooms,total_area,living_area,kitchen_area,period,p_income,p_outcome
0,Bulvar Rokossovskogo,35000,4,transport,agency,50,9,6,9,2,40,30,7,0.0,1650435.0,1655224.0
1,Bulvar Rokossovskogo,35000,4,transport,agency,50,9,6,9,2,40,30,7,1.0,1784777.0,1789582.0
2,Bulvar Rokossovskogo,35000,4,transport,agency,50,9,6,9,2,40,30,7,0.0,774605.0,808964.0
3,Bulvar Rokossovskogo,35000,4,transport,agency,50,9,6,9,2,40,30,7,1.0,869366.0,911094.0
4,Bulvar Rokossovskogo,35000,4,transport,agency,50,9,6,9,2,40,30,7,2.0,845177.0,891599.0


In [20]:
# а здесь придётся немного подзапариться, чтобы сохранить целостность тестового набора данных
last_period = metro.groupby('NameOfStation') \
                   .agg({'period': 'max'}) \
                   .reset_index()
current_metro = metro.merge(last_period) \
                     .groupby(['NameOfStation', 'period']) \
                     .agg({'p_income': 'mean', 'p_outcome': 'mean'}) \
                     .reset_index()
tmp = test_X.merge(en_ru, how='left').merge(current_metro, how='left').drop(columns='NameOfStation')
print(f'len(test_X) = {len(test_X)}, len(tmp) = {len(tmp)}.')

len(test_X) = 849, len(tmp) = 849.


In [21]:
test_X = tmp
test_X.head()

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,metro,minutes,way,provider,fee_percent,views,floor,total_floors,rooms,total_area,living_area,kitchen_area,period,p_income,p_outcome
0,Kommunarka,11,walk,agency,60,7,7,15,2,58,35,11,18.0,517226.0,523637.0
1,No data,0,walk,agency,20,32,10,14,1,64,64,64,NaN,NaN,NaN
2,Salarevo,16,transport,agency,100,35,11,16,1,64,64,64,18.0,3495100.0,3262187.0
3,Prospekt Vernadskogo,16,walk,developer,50,19,1,7,2,58,36,9,18.0,1218821.0,1249928.0
4,No data,0,walk,agency,0,16,2,12,7,64,64,64,NaN,NaN,NaN


## 3

In [22]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60682 entries, 0 to 60681
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   metro         60682 non-null  object 
 1   price         60682 non-null  int64  
 2   minutes       60682 non-null  int64  
 3   way           60682 non-null  object 
 4   provider      60682 non-null  object 
 5   fee_percent   60682 non-null  int64  
 6   views         60682 non-null  int64  
 7   floor         60682 non-null  int64  
 8   total_floors  60682 non-null  int64  
 9   rooms         60682 non-null  int64  
 10  total_area    60682 non-null  int64  
 11  living_area   60682 non-null  int64  
 12  kitchen_area  60682 non-null  int64  
 13  period        59902 non-null  float64
 14  p_income      59902 non-null  float64
 15  p_outcome     59902 non-null  float64
dtypes: float64(3), int64(10), object(3)
memory usage: 7.4+ MB


In [23]:
test_X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 849 entries, 0 to 848
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   metro         849 non-null    object 
 1   minutes       849 non-null    int64  
 2   way           849 non-null    object 
 3   provider      849 non-null    object 
 4   fee_percent   849 non-null    int64  
 5   views         849 non-null    int64  
 6   floor         849 non-null    int64  
 7   total_floors  849 non-null    int64  
 8   rooms         849 non-null    int64  
 9   total_area    849 non-null    int64  
 10  living_area   849 non-null    int64  
 11  kitchen_area  849 non-null    int64  
 12  period        673 non-null    float64
 13  p_income      673 non-null    float64
 14  p_outcome     673 non-null    float64
dtypes: float64(3), int64(9), object(3)
memory usage: 99.6+ KB


In [24]:
columns = ['period', 'p_income', 'p_outcome']
for c in columns:
    train_c_median = train[c].median()
    train[c] = train[c].fillna(train_c_median)
    test_X[c] = test_X[c].fillna(train_c_median)
print('done.')

done.


## 4

In [25]:
print(f'Unique stations in train = {len(train.metro.unique())}.')
print(f'Unique stations in test_X = {len(test_X.metro.unique())}.')

Unique stations in train = 212.
Unique stations in test_X = 175.


### Из-за различия чисел выше, для 'metro' придётся реализовывать свой get_dummies

In [26]:
stations = sorted(train['metro'].unique())
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    for station in stations:
        train[station] = (train['metro'] == station).astype(int)
        test_X[station] = (test_X['metro'] == station).astype(int)
train = train.drop(columns='metro')
test_X = test_X.drop(columns='metro')
print('done.')

done.


In [27]:
train = pd.concat([
    train,
    pd.get_dummies(train['way'], dtype=int),
    pd.get_dummies(train['provider'], dtype=int)
], axis=1).drop(columns=['way', 'provider'])
test_X = pd.concat([
    test_X,
    pd.get_dummies(test_X['way'], dtype=int),
    pd.get_dummies(test_X['provider'], dtype=int)
], axis=1).drop(columns=['way', 'provider'])
print('done.')

done.


In [28]:
train.head()

,price,minutes,fee_percent,views,floor,total_floors,rooms,total_area,living_area,kitchen_area,...,Vykhino,Vystavochnaia,Zhulebino,Ziablikovo,transport,walk,agency,developer,owner,realtor
0,35000,4,50,9,6,9,2,40,30,7,...,0,0,0,0,1,0,1,0,0,0
1,35000,4,50,9,6,9,2,40,30,7,...,0,0,0,0,1,0,1,0,0,0
2,35000,4,50,9,6,9,2,40,30,7,...,0,0,0,0,1,0,1,0,0,0
3,35000,4,50,9,6,9,2,40,30,7,...,0,0,0,0,1,0,1,0,0,0
4,35000,4,50,9,6,9,2,40,30,7,...,0,0,0,0,1,0,1,0,0,0


In [29]:
test_X.head()

,minutes,fee_percent,views,floor,total_floors,rooms,total_area,living_area,kitchen_area,period,...,Vykhino,Vystavochnaia,Zhulebino,Ziablikovo,transport,walk,agency,developer,owner,realtor
0,11,60,7,7,15,2,58,35,11,18.0,...,0,0,0,0,0,1,1,0,0,0
1,0,20,32,10,14,1,64,64,64,9.0,...,0,0,0,0,0,1,1,0,0,0
2,16,100,35,11,16,1,64,64,64,18.0,...,0,0,0,0,1,0,1,0,0,0
3,16,50,19,1,7,2,58,36,9,18.0,...,0,0,0,0,0,1,0,1,0,0
4,0,0,16,2,12,7,64,64,64,9.0,...,0,0,0,0,0,1,1,0,0,0


## 5

In [30]:
sum(train['minutes'] > 60)

0

In [31]:
sum(train['fee_percent'] > 100)

0

In [32]:
sum(train['floor'] > train['total_floors'])

37

In [33]:
sum(train['total_floors'] > 123)

223

In [34]:
sum(train['rooms'] > 10)

0

In [35]:
sum(train['total_area'] > 500)

0

In [36]:
sum(train['living_area'] + train['kitchen_area'] > train['total_area'])

24125

In [37]:
sum((train['total_area'] == train['living_area']) | (train['living_area'] == train['kitchen_area']) | (train['total_area'] == train['kitchen_area']))

21389

In [38]:
train = train[(train['floor'] <= train['total_floors']) & (train['total_floors'] <= 123)]
print('done.')

done.


In [39]:
train['flag1'] = (train['living_area'] + train['kitchen_area'] > train['total_area']).astype(int)
train['flag2'] = (train['total_area'] == train['living_area']).astype(int)
train['flag3'] = (train['living_area'] == train['kitchen_area']).astype(int)
train['flag4'] = (train['total_area'] == train['kitchen_area']).astype(int)
train['flag5'] = (train['flag2'] + train['flag3'] + train['flag4'] == 3).astype(int)

test_X['flag1'] = (test_X['living_area'] + test_X['kitchen_area'] > test_X['total_area']).astype(int)
test_X['flag2'] = (test_X['total_area'] == test_X['living_area']).astype(int)
test_X['flag3'] = (test_X['living_area'] == test_X['kitchen_area']).astype(int)
test_X['flag4'] = (test_X['total_area'] == test_X['kitchen_area']).astype(int)
test_X['flag5'] = (test_X['flag2'] + test_X['flag3'] + test_X['flag4'] == 3).astype(int)

print('done.')

done.


## 6

In [40]:
columns_to_normalize = []
for c in train.columns:
    if c == 'price':
        continue
    if len(train[c].unique()) > 2:
        columns_to_normalize.append(c)
columns_to_normalize

['minutes',
 'fee_percent',
 'views',
 'floor',
 'total_floors',
 'rooms',
 'total_area',
 'living_area',
 'kitchen_area',
 'period',
 'p_income',
 'p_outcome']

In [41]:
normalizer = Normalizer().fit(train[columns_to_normalize])
train[columns_to_normalize] = normalizer.transform(train[columns_to_normalize])
test_X[columns_to_normalize] = normalizer.transform(test_X[columns_to_normalize])
print('done.')

done.


In [42]:
train.head()

,price,minutes,fee_percent,views,floor,total_floors,rooms,total_area,living_area,kitchen_area,...,walk,agency,developer,owner,realtor,flag1,flag2,flag3,flag4,flag5
0,35000,0.000002,0.000021,0.000004,0.000003,0.000004,8.556310e-07,0.000017,0.000013,0.000003,...,0,1,0,0,0,0,0,0,0,0
1,35000,0.000002,0.000020,0.000004,0.000002,0.000004,7.913096e-07,0.000016,0.000012,0.000003,...,0,1,0,0,0,0,0,0,0,0
2,35000,0.000004,0.000045,0.000008,0.000005,0.000008,1.785689e-06,0.000036,0.000027,0.000006,...,0,1,0,0,0,0,0,0,0,0
3,35000,0.000003,0.000040,0.000007,0.000005,0.000007,1.588157e-06,0.000032,0.000024,0.000006,...,0,1,0,0,0,0,0,0,0,0
4,35000,0.000003,0.000041,0.000007,0.000005,0.000007,1.627969e-06,0.000033,0.000024,0.000006,...,0,1,0,0,0,0,0,0,0,0


In [43]:
test_X.head()

,minutes,fee_percent,views,floor,total_floors,rooms,total_area,living_area,kitchen_area,period,...,walk,agency,developer,owner,realtor,flag1,flag2,flag3,flag4,flag5
0,0.000015,0.000082,0.000010,9.510672e-06,0.000020,2.717335e-06,0.000079,0.000048,0.000015,0.000024,...,1,1,0,0,0,0,0,0,0,0
1,0.000000,0.000007,0.000011,3.295847e-06,0.000005,3.295847e-07,0.000021,0.000021,0.000021,0.000003,...,1,1,0,0,0,1,1,1,1,1
2,0.000003,0.000021,0.000007,2.300793e-06,0.000003,2.091630e-07,0.000013,0.000013,0.000013,0.000004,...,0,1,0,0,0,1,1,1,1,1
3,0.000009,0.000029,0.000011,5.728008e-07,0.000004,1.145602e-06,0.000033,0.000021,0.000005,0.000010,...,1,0,1,0,0,0,0,0,0,0
4,0.000000,0.000000,0.000005,6.591693e-07,0.000004,2.307093e-06,0.000021,0.000021,0.000021,0.000003,...,1,1,0,0,0,1,1,1,1,1


## 7

In [44]:
train_y = train['price']
train_X = train.drop(columns='price')
print('done.')

done.


In [45]:
class ThreeLayerPerceptron(nn.Module):
    def __init__(self,
                 inputlayer_size:   int,
                 hiddenlayer1_size: int,
                 hiddenlayer2_size: int,
                 hiddenlayer3_size: int,
                 outputlayer_size:  int) -> None:
        
        super().__init__()

        self.fc1 = nn.Linear(inputlayer_size, hiddenlayer1_size)
        self.fc2 = nn.Linear(hiddenlayer1_size, hiddenlayer2_size)
        self.fc3 = nn.Linear(hiddenlayer2_size, hiddenlayer3_size)
        self.out = nn.Linear(hiddenlayer3_size, outputlayer_size)

        return

    def forward(self,
                x: torch.Tensor) -> torch.Tensor:
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.out(x)
        return x

print('\'ThreeLayerPerceptron\' class has been successfully defined.')

'ThreeLayerPerceptron' class has been successfully defined.


In [46]:
model = ThreeLayerPerceptron(len(train_X.columns), 128, 64, 32, 1).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
print('done.')

done.


In [47]:
train_X = train_X.values
train_y = train_y.values
test_X = test_X.values
# сейчас они все стали np.ndarray-ами
print('done.')

done.


In [48]:
train_X = torch.FloatTensor(train_X)
train_y = torch.FloatTensor(train_y).view(-1, 1)
test_X = torch.FloatTensor(test_X)
# сейчас они все стали torch.tensor-ами
print('done.')

done.


In [49]:
BATCH_SIZE = 256

dataset = TensorDataset(train_X, train_y)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

print('done.')

done.


In [50]:
EPOCHS = 500

models = []
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    for x, y in dataloader:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()

        outs = model(x)
        loss = criterion(outs, y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * len(y)
    train_loss = total_loss / len(train)

    if (epoch + 1) % 10 == 0:
        print(f'Epoch {epoch + 1:02d} | Train loss = {train_loss:.4f}')
    if (epoch + 1) % 100 == 0:
        models.append(deepcopy(model))

Epoch 10 | Train loss = 39144330370.5173
Epoch 20 | Train loss = 35355526827.0223
Epoch 30 | Train loss = 34953038423.6749
Epoch 40 | Train loss = 34381667254.9757
Epoch 50 | Train loss = 33625646135.5029
Epoch 60 | Train loss = 32795366704.1395
Epoch 70 | Train loss = 32031368073.6944
Epoch 80 | Train loss = 31515345166.9936
Epoch 90 | Train loss = 31164093126.4774
Epoch 100 | Train loss = 30938099646.0808
Epoch 110 | Train loss = 30771762001.7681
Epoch 120 | Train loss = 30626786368.8860
Epoch 130 | Train loss = 30522958792.4633
Epoch 140 | Train loss = 30446516918.0484
Epoch 150 | Train loss = 30354989639.5508
Epoch 160 | Train loss = 30294531907.3207
Epoch 170 | Train loss = 30231998189.6868
Epoch 180 | Train loss = 30181588847.7637
Epoch 190 | Train loss = 30130860280.9753
Epoch 200 | Train loss = 30096835737.3993
Epoch 210 | Train loss = 30067547819.0732
Epoch 220 | Train loss = 30027593742.4982
Epoch 230 | Train loss = 30022930662.5478
Epoch 240 | Train loss = 29977674755.4298
E

## 8

In [51]:
len(models)

5

In [52]:
sub = pd.read_csv('/kaggle/input/flat-rent-prediction/sample_submission.csv')
sub.head()

,Unnamed: 0,0
0,1350,0
1,1434,0
2,2500,0
3,1128,0
4,4145,0


In [53]:
sub = sub.rename(columns={'Unnamed: 0': 'ID'})
sub.head()

,ID,0
0,1350,0
1,1434,0
2,2500,0
3,1128,0
4,4145,0


In [54]:
predict = models[0](test_X.to(device)).cpu().detach().numpy().reshape(len(sub))
test_X = test_X.to(device)
for i, model in enumerate(models, 1):
    predict = model(test_X).cpu().detach().numpy().reshape(len(sub))
    sub['0'] = predict
    sub.to_csv(f'sub{i}.csv', index=False)
print('done.')

done.


# Вывод
## 1. Нейронки - пока что не мой уровень;
## 2. Не всё решается нейронками (самые нормальные submit-ы у меня - это бустинги).